# Analysis of GitHub Activity Data

For this hackathon, House Star Wars used [GitHub Activity Data](https://console.cloud.google.com/marketplace/details/github/github-repos?q=github&id=46ee22ab-2ca4-4750-81a7-3ee0f0150dcb), a public dataset from Google BigQuery, to look at the trends of programming languages related to data science.

### Which programming languages do data scientists use?

We did some research on the [top data science programming languages](https://www.analyticsinsight.net/top-10-data-science-programming-languages-for-2020/) to come up with our list below (in no particular order):

- Python
- R
- Jupyter Notebook
- SQL
- Matlab
- MATLAB
- JavaScript
- Java
- C++
- Julia
- Swift

We also included Jupyter Notebook since it's listed on GitHub's top languages for its [data-science topic](https://github.com/topics/data-science), even though technically it's not a programming language.

<img src="figs/Screenshot from 2020-05-30 16-29-42.png">

#### Plot number of commits for data science programming languages

In [ ]:
import google.datalab.bigquery as bq
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

```sql
-- count of repos per language, in desc order
SELECT lang.name, COUNT(repo_name)
FROM `bigquery-public-data.github_repos.languages`, UNNEST(language) as lang
WHERE lang.name IN ('Python', 'R', 'Jupyter Notebook', 'SQL', 'Matlab',
    'MATLAB', 'JavaScript', 'Java', 'C++', 'Julia', 'Swift')
GROUP BY lang.name
ORDER BY COUNT(repo_name) DESC
```

```sql
-- number of repos per license, in desc order
SELECT license, count(repo_name)
FROM `bigquery-public-data.github_repos.licenses`
GROUP BY license
ORDER BY count(repo_name) DESC
```

#### Plot Counts of Repos per Language

In [ ]:
%%bq query -n languages

SELECT lang.name, COUNT(repo_name)
FROM `bigquery-public-data.github_repos.languages`, UNNEST(language) as lang
WHERE lang.name IN ('Python', 'R', 'Jupyter Notebook', 'SQL',
'Matlab', 'MATLAB', 'Javascript', 'Java',
'C++', 'Julia', 'Swift')
GROUP BY lang.name

UsageError: Cell magic `%%bq` not found.


In [ ]:
%%bq sample --query languages

In [ ]:
languages_df = languages.execute(output_options=bq.QueryOutput.dataframe()).result()

In [ ]:
languages_df['count'] = languages_df['f0_']

In [ ]:
languages_df = languages_df.sort_values('count',ascending=False)

ax = languages_df.plot(kind='bar', color='#ffc39fff', figsize=(16,8), fontsize=12, rot=0)
ax.grid('on', which='major', axis='y', linestyle='--', alpha=0.5)
plt.xlabel('Languages', size=12, labelpad=5)
plt.ylabel('Count', size=12, labelpad=5)
plt.title('Data Science Languages', fontsize=20)
ax.set_facecolor('#434343')
ax.get_legend().remove()
plt.show()

### Plot trend line for data science programming languages

#### Join languages & sample_commits table to look for repo_name that uses DS programming languages and the dates of commit

In [ ]:
# Create query - commits table
qry_commits = bq.Query('''

WITH lang AS (
  SELECT repo_name, language as l
  FROM
    `bigquery-public-data.github_repos.languages`)

SELECT l.name, com.author.date
FROM lang CROSS JOIN lang.l
RIGHT JOIN `bigquery-public-data.github_repos.sample_commits` as com
ON lang.repo_name = com.repo_name
WHERE com.repo_name IN (
  SELECT repo_name
  FROM lang, lang.l
  WHERE l.name IN ('Python', 'R', 'Jupyter Notebook', 'SQL', 'Matlab', 'MATLAB', 'JavaScript', 'Java', 'C++','Julia', 'Swift'))

''')

In [ ]:
# Run query
result_commits = qry_commits.execute().result()

In [ ]:
# Convert to dataframe
df_commits = result_commits.to_dataframe()

In [ ]:
lang = ['Python', 'R', 'Jupyter Notebook', 'MATLAB', 'JavaScript', 'Java', 'C++','Swift']

#### Plot monthly trend

In [ ]:
plt.figure(figsize=(15, 9))  
ax = plt.subplot(111)  

for i,name in enumerate (lang):  
    # Plot each line separately with its own color
    data = df_commits[df_commits['name']==name].resample('M').count()
    data.plot(ax=ax, color='C'+str(i), label = name)
    
plt.ylabel('Number of Commits')
plt.title("Commits over time per language") 
plt.legend(lang)

### Additonal: Explore languages & bytes distribution

#### Language pie chart

In [ ]:
# Create query - languages table
qry_lang = bq.Query('''

WITH lang AS (
  SELECT language as l
  FROM
    `bigquery-public-data.github_repos.languages`)
    
SELECT l.name, l.bytes
FROM lang, lang.l

''')

In [ ]:
# Run query
result_lang = qry_lang.execute().result()

In [ ]:
df_lang_count = df_lang.groupby('name')['name'].count().sort_values(ascending=False)
df_lang_count = pd.DataFrame(df_lang_count)
df_lang_count.columns = ['count']
df_lang_count = df_lang_count.reset_index()
df_lang_count.loc[10:,'name'] = 'Others'
df_lang_count = df_lang_count.groupby('name')['count'].sum().sort_values(ascending=False)
df_lang_count = pd.DataFrame(df_lang_count)
df_lang_count.columns = ['count']
df_lang_count = df_lang_count.reset_index()
df_lang_count

In [ ]:
fig, ax = plt.subplots()

def my_autopct(pct):
    return ('%1.0f%%' % pct) if pct > 3 else ''

ax.pie(df_lang_count['count'], autopct=my_autopct, radius=2.5, rotatelabels=0,
        pctdistance=0.8, startangle=90, colors=['#EFFCEE','#D46288', '#94D3A9', '#F7E09C', '#FEF2F2', '#6D937A', '#85D3C5','#CFAE82', '#D67FA1', '#FFEEDB', '#ADA8B6', '#4C3B4D'])
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

my_circle=plt.Circle( (0,0), 1.5, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

#plt.savefig('types.png', title='Types of Traffic Incidence along EDSA')
plt.title('% Language', fontdict= {'fontsize': 15})
plt.legend(df_lang_count['name'], loc='upper right', bbox_to_anchor=(1.15,0.9))
plt.show()

#### Bytes pie chart

In [ ]:
df_lang_bytes = df_lang.groupby('name')['bytes'].sum().sort_values(ascending=False)
df_lang_bytes = pd.DataFrame(df_lang_bytes)
df_lang_bytes.columns = ['bytes']
df_lang_bytes = df_lang_bytes.reset_index()

In [ ]:
df_lang_bytes.loc[10:,'name'] = 'Others'
df_lang_bytes = df_lang_bytes.groupby('name')['bytes'].sum().sort_values(ascending=False)
df_lang_bytes = pd.DataFrame(df_lang_bytes)
df_lang_bytes.columns = ['bytes']
df_lang_bytes = df_lang_bytes.reset_index()
df_lang_bytes

In [ ]:
fig, ax = plt.subplots()

ax.pie(df_lang_bytes['bytes'], autopct=my_autopct, radius=2.5, rotatelabels=0, labeldistance=1.1,textprops={'fontsize': 13},
        pctdistance=0.8, startangle=90, colors=['#EFFCEE','#D46288', '#94D3A9', '#F7E09C', '#FEF2F2', '#6D937A', '#85D3C5','#CFAE82', '#D67FA1', '#FFEEDB', '#ADA8B6', '#4C3B4D'])
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

my_circle=plt.Circle( (0,0), 1.5, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

#plt.savefig('types.png', title='Types of Traffic Incidence along EDSA')
plt.title('% Bytes', fontdict= {'fontsize': 15})
plt.legend(df_lang_bytes['name'], loc='upper right', bbox_to_anchor=(1.15,0.9))
plt.show()